# Introduction To Ansible

Ansible is a tool for automating system administration and operation tasks. It uses SSH to talk to a collection of servers and do tasks such as: configure aspects of the server, install or upgrade software, and pretty much any other task you could do at the command line.

## Basic Notions

Ansible does not run any agents on the remote servers. Instead, it connects over SSH to each server and runs a (typically small) program to accomplish each task. Users write Ansible scripts to perform multiple tasks on a given set of servers. Users then execute these scripts from any machine that has access to Ansible and 

Make sure Ansible is installed on your VM:
```
$ ansible --version

```
If not, install it with:
```
$ sudo apt-get install ansible
```

### Hosts Files
Ansible has the notion of a hosts or inventory file. This is a plain text file that describes all of the servers you want Ansible to interact with. Each host listed in the file should provide the basic SSH connectivity information for Ansible to use to perform tasks on them. Hosts can be also be collected into groups.

Here is an example hosts file with three hosts, two in the "web" group and one in the "databases" group.

```
[web]
web_1 ansible_ssh_host=129.114.18.27 ansible_ssh_private_key_file=~/web.key ansible_ssh_user=ubuntu
web_2 ansible_ssh_host=129.114.18.28 ansible_ssh_private_key_file=~/web.key ansible_ssh_user=ubuntu

[databases]
mysql ansible_ssh_host=129.114.19.33 ansible_ssh_private_key_file=~/db.key ansible_ssh_user=centos

```

Note that each line corresponding to a host begins with a name, which can be any identifier we want, and then provides two variables `ansible_ssh_host` and `ansible_ssh_private_key_file`. 

```
Exercise. Create a hosts file for your jupyter host. Don't worry about putting it in a group yet.
```

Once we have a hosts file, we can already run basic commands against the host(s). To run a command on all hosts in a hosts file, use the following:

```
$ ansible all -i <hosts_file> -a '<ommand>'
```

For example, let's check the uptime of the server by running the "uptime" command:
```
$ ansible all -i hosts -a 'uptime'
```

### Modules and Tasks

Part of Ansible's power comes from its rich library of modules. Modules are configurable programs that can be used so accomplish specific tasks. For example, we have the "command" module for simply running arbitrary commands. But there are many other modules for describing tasks at a higher level. For example, Ansible provides the "copy" module for ensuring files from the Ansible host machine are present on the remote server.

We can specify a module to use by passing the name of the module to the `-m` flag and then providing any required parameters of the module. For instance, the copy module requires `src` and `dest` parameters. Let's use the copy module to copy a file called "test.txt" in the current directory to `/root` in the remote server. We'll name it "remote.txt" on the remote:
```
$ ansible all -m copy -a "src=test.txt dest=/root/remote.txt"
10.10.100.7 | SUCCESS => {
    "changed": true, 
    "checksum": "da39a3ee5e6b4b0d3255bfef95601890afd80709", 
    "dest": "/root/remote.txt", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "d41d8cd98f00b204e9800998ecf8427e", 
    "mode": "0644", 
    "owner": "root", 
    "size": 0, 
    "src": "/root/.ansible/tmp/ansible-tmp-1500597877.29-186213310577275/source", 
    "state": "file", 
    "uid": 0
}
```

After running that command we see some output (note the color). In particular, we see a `"changed": true` . Ansible detected that we it needed to actually change the host to ensure that the file was there. Let's try running the command again and see what we get:
```
$ ansible all -m copy -a "src=test.txt dest=/root/remote.txt"
10.10.100.7 | SUCCESS => {
    "changed": false, 
    "checksum": "da39a3ee5e6b4b0d3255bfef95601890afd80709", 
    "dest": "/root/remote.txt", 
    "gid": 0, 
    "group": "root", 
    "mode": "0644", 
    "owner": "root", 
    "path": "/root/remote.txt", 
    "size": 0, 
    "state": "file", 
    "uid": 0
}
```
This time, the output is green and Ansible indicates to us that it didn't change anything on the remote server. Indeed, Ansible's copy module first checks if there is anything to do, and if the file already there, it doesn't do anything. This is a key part of Ansible's power which we explore in the next section. But first, let's change the contents of our test.txt and then re-run our command.

```
# open test.txt in vi and change the contents:
$ vi test.txt
# . . .
# now run the command:
$ ansible all -m copy -a "src=test.txt dest=/root/remote.txt"
10.10.100.9 | SUCCESS => {
    "changed": true, 
    "checksum": "4e1243bd22c66e76c2ba9eddc1f91394e57f9f83", 
    "dest": "/root/remote.txt", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "d8e8fca2dc0f896fd7cb4cb0031ba249", 
    "mode": "0644", 
    "owner": "root", 
    "size": 5, 
    "src": "/root/.ansible/tmp/ansible-tmp-1500598336.43-267287598231809/source", 
    "state": "file", 
    "uid": 0
}
```

### Idempotence
Ansible's modules provide us with the power to ensure a given Ansible script we write is `idempotent`. A given task is said to be idempotent if performing the task once produces the same result as performing it repeadly, assuming no other intervening actions.

Idempotence is very important for ensuring re-runability of your provisioning scripts. Suppose you wanted to automate maintenance of a database sever. You might have the following high level tasks:

```
1. Create a linux user account for the MySQL service to run under
2. Install the MySQL server package
3. Start the mysql daemon
4. Create the database
5. Run the latest migrations on the database to install the schema
```

The problem is that, if we just use basic commands such as `useradd` and `mysql create database`, the script will run on a brand new server but it will fail every subsequent time. With idempotent tasks we can avoid this issue. 

```
1. Ensure a linux user account exists for the MySQL service to run under
2. Ensure the MySQL server package is installed
3. Ensure the mysql daemon is running
4. Ensure the database has been created
5. Ensure the database has the latest migrations on the database to install the schema.
```


